In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep


driver = webdriver.Firefox()
driver.get("https://www.imdb.com/search/title/?title_type=feature&release_date=1950-01-18,&sort=num_votes,desc")
sleep(3)


In [ ]:
import time

PAGE_TO_COUNT = 123

def load_movie_links(driver):
    page_counter = 0
    while(page_counter < PAGE_TO_COUNT):
        more_button = driver.find_element(By.CLASS_NAME, 'ipc-see-more__text')
        more_button.click()
        time.sleep(4)
        page_counter += 1
    
load_movie_links(driver)


In [5]:
from bs4 import BeautifulSoup
def fetch_movie_links(driver):
    link_list = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    movie_links = soup.find_all('a', {'class' : 'ipc-title-link-wrapper'})
    for link in movie_links:
        link_list.append(link['href'])
    return link_list

movie_links = fetch_movie_links(driver)



In [6]:
with open("links.txt", 'w') as file:
    [file.write(link + "\n") for link in movie_links]

In [7]:
def get_movie_info(soup):
    title = soup.find('span', {'class': 'hero__primary-text'}).text.strip()
    year_and_dur_and_cert = soup.find('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt'}).find_all('li', {'class' : 'ipc-inline-list__item'})
    
    release_year = year_and_dur_and_cert[0].text
    cert = year_and_dur_and_cert[1].text if len(year_and_dur_and_cert) >= 2 else 'N/A'
    duration = year_and_dur_and_cert[-1].text
    genre = [g.text for g in soup.find('div', attrs={'data-testid': 'genres'}).find_all('span', {'class' : 'ipc-chip__text'})]
    rating = soup.find('span', {'class': 'sc-bde20123-1 cMEQkK'}).text
    votes = soup.find('div', {'class': 'sc-bde20123-3 gPVQxL'}).text
    meta_score = soup.find('span', {'class': 'sc-b0901df4-0 bcQdDJ metacritic-score-box'}).text if soup.find('span', {'class': 'sc-b0901df4-0 bcQdDJ metacritic-score-box'}) else 'N/A'
    language = [l.text for l in soup.find('li', {'data-testid' : 'title-details-languages'}).find_all('a', {'class' : 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})]
    countries_of_origin = [c.text for c in soup.find('li', {'data-testid' : 'title-details-origin'}).find_all('a', {'class' : 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})]
    budget = soup.find('li', {'data-testid' : 'title-boxoffice-budget'}).find('span', {'class' : 'ipc-metadata-list-item__list-content-item'}).text.removesuffix(' (estimated)') if soup.find('li', {'data-testid' : 'title-boxoffice-budget'}) else 'N/A'
    gross = soup.find('li', {'data-testid' : 'title-boxoffice-cumulativeworldwidegross'}).find('span', {'class' : 'ipc-metadata-list-item__list-content-item'}).text.removesuffix(' (estimated)') if soup.find('li', {'data-testid' : 'title-boxoffice-cumulativeworldwidegross'}) else 'N/A'
    company = [c.text for c in soup.find('li', {'data-testid' : 'title-details-companies'}).find_all('li', {'class' : 'ipc-inline-list__item'})]
    return [title, release_year, cert, duration, genre, rating, votes, meta_score, language, countries_of_origin, budget, gross, company]


In [8]:
titles = []
release_years = []
certs = []
durations = []
genres = []
ratings = []
num_votes = []
meta_scores = []
languages = []
countries = []
budgets = []
grosses = []
producer_companies = []

In [ ]:
from time import sleep
IMDB_LINK = "https://www.imdb.com"

## index 404 had no gross
for link in movie_links:
    movie_link = IMDB_LINK + link
    driver.get(movie_link)
   # sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    movie_info = get_movie_info(soup)
    titles.append(movie_info[0])
    release_years.append(movie_info[1])
    certs.append(movie_info[2])
    durations.append(movie_info[3])
    genres.append('~'.join(movie_info[4]))
    ratings.append(movie_info[5])
    num_votes.append(movie_info[6])
    meta_scores.append(movie_info[7])
    languages.append('~'.join(movie_info[8]))
    countries.append('~'.join(movie_info[9]))
    budgets.append(movie_info[10])
    grosses.append(movie_info[11])
    producer_companies.append('~'.join(movie_info[12]))

In [ ]:
import pandas as pd
print(len(titles))
print(len(release_years))
print(len(certs))
print(len(durations))
print(len(genres))
print(len(ratings))
print(len(num_votes))
print(len(meta_scores))
print(len(languages))
print(len(countries))
print(len(producer_companies))
print(len(budgets))

df = pd.DataFrame({
    'Title': titles,
    'Release Year': release_years,
    'Age Restriction':certs,
    'Duration': durations,
    'Genre': genres,
    'Rating': ratings,
    'Number of Votes': num_votes,
    'Meta Score': meta_scores,
    'Language': languages,
    'Country of Origin': countries,
    'Companies' : producer_companies,
    'Budget': budgets,
    'Gross (worldwide)': grosses
    
})

df.to_csv("imdb_dataset.csv")